In [1]:
import torch
import torch.optim as optim

from utils.MNIST_loaders import MNIST_loaders

from functions.train import train
from functions.evaluate import evaluate

from models.HypernetClassifier import HyperNetClassifier
from models.Classifier import Classifier

In [ ]:
# --- Hyperparameters (things you can easily change!) ---
num_epochs = 5
batch_size = 64
weight_decay = 0.001
validation_split = 0.2  # Part of the training data to use for validation
random_seed = 42
n_bins = 10 # Number of bins for calculating ECE
input_size = 28 * 28
output_size = 10

learning_rate = 0.0001
optimizer = 'Adam'
hidden_size = [256, 128, 256]
zeroed_weights_in_baseline = 0.0 #! Here we don't zero out any weights
hypernet_ensemble_num = 1
use_previous_weights = True

In [11]:
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
model = Classifier(input_size, output_size, zeroed_weights_fraction=zeroed_weights_in_baseline)
hypernet_model = HyperNetClassifier(input_size, output_size, hidden_sizes=hidden_size, device=device, use_previous_weights=use_previous_weights, ensemble_num=hypernet_ensemble_num)

if optimizer == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    hypernet_optimizer = optim.SGD(hypernet_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
else:
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    hypernet_optimizer = optim.Adam(hypernet_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
train_loader, val_loader, test_loader = MNIST_loaders()

print("Starting Training [A: Accuracy, L: Loss, ECE: Expected Calibration Error, H: Hypernet]")
for epoch in range(1, num_epochs + 1):
    
    print(f"\n=== Epoch: [{epoch}|{num_epochs}] ========================")
    
    avg_loss, avg_hypernet_loss, accuracy, hypernet_accuracy = train(model, hypernet_model, train_loader, optimizer, hypernet_optimizer, device=device)
    print(f'Train: L: {avg_loss:.4f}, H_L: {avg_hypernet_loss:.4f}, A: {accuracy:.2f}%, H_A: {hypernet_accuracy:.2f}%')
    
    val_accuracy, ece = evaluate(model, val_loader, n_bins=n_bins, device=device)
    hypernet_val_accuracy, hypernet_ece = evaluate(hypernet_model, val_loader, device=device)
    print(f'Val: A: {val_accuracy:.2f}%, A_H: {hypernet_val_accuracy:.2f}%, ECE: {ece:.5f}, ECE_H: {hypernet_ece:.5f}')

Starting Training [A: Accuracy, L: Loss, ECE: Expected Calibration Error, H: Hypernet]

=== Epoch: [1|5] ========================
Train: L: 0.9092, H_L: 0.6470, A: 77.10%, H_A: 83.42%
Val: A: 86.86%, A_H: 89.49%, ECE: 0.15420, ECE_H: 0.00828

=== Epoch: [2|5] ========================
Train: L: 0.4563, H_L: 0.3359, A: 88.10%, H_A: 90.24%
Val: A: 88.98%, A_H: 90.11%, ECE: 0.09058, ECE_H: 0.00530

=== Epoch: [3|5] ========================
Train: L: 0.3820, H_L: 0.3185, A: 89.61%, H_A: 90.78%
Val: A: 89.91%, A_H: 90.61%, ECE: 0.06662, ECE_H: 0.01017

=== Epoch: [4|5] ========================
Train: L: 0.3486, H_L: 0.3092, A: 90.34%, H_A: 91.25%
Val: A: 90.57%, A_H: 91.33%, ECE: 0.05517, ECE_H: 0.01752

=== Epoch: [5|5] ========================
Train: L: 0.3290, H_L: 0.2997, A: 90.79%, H_A: 91.31%
Val: A: 90.75%, A_H: 90.91%, ECE: 0.04549, ECE_H: 0.01498


In [13]:
# --- Testing the Model ---
print("\nStarting Testing...")

print("Normal Model:")
test_accuracy, ece = evaluate(model, test_loader, n_bins=n_bins, device=device)
print(f'Test set: Accuracy: {test_accuracy:.2f}%, ECE: {ece:.5f}')

print("Hypernet Model:")
test_accuracy, ece = evaluate(hypernet_model, test_loader, n_bins=n_bins, device=device)
print(f'Test set: Accuracy: {test_accuracy:.2f}%, ECE: {ece:.5f}')


Starting Testing...
Normal Model:
Test set: Accuracy: 91.31%, ECE: 0.04334
Hypernet Model:
Test set: Accuracy: 91.07%, ECE: 0.01071
